In [1]:
import openai
import numpy as np
import faiss
import json
import os

# Set your OpenAI API key

client = openai.OpenAI()
from dotenv import load_dotenv
# Set your OpenAI API key

load_dotenv()

openai.api_key = os.getenv('OPENAI_API_KEY')

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

file_path = 'articles_topics.json'
data = load_json(file_path)

# List of unique topics
topics = list(set(article['metadata']['type'] for article in data))

# Function to generate embeddings using the OpenAI API
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding
    #response = openai.embeddings.create(input=[text], model=model)
    #return response['data'][0]['embedding']

# Generate embeddings for each topic
topic_embeddings = {topic: get_embedding(topic) for topic in topics}

# Convert embeddings to NumPy array
embeddings = np.array([topic_embeddings[topic] for topic in topics]).astype('float32')

# Create a FAISS index for the topics
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


In [2]:
def find_relevant_topic(query):
    query_embedding = np.array(get_embedding(query)).astype('float32').reshape(1, -1)
    distances, indices = index.search(query_embedding, 1)
    return topics[indices[0][0]]

# Example user query
user_query = "Kann ich gekündet werden, wenn ich invalide bin?"
relevant_topic = find_relevant_topic(user_query)
print(f"Relevant topic: {relevant_topic}")


Relevant topic: Rechte und Pflichten der Angestellten 


In [3]:
relevant_articles = [article for article in data if article['metadata']['type'] == relevant_topic]

# Generate embeddings for these articles
for article in relevant_articles:
    article_text = article['text']
    article_tags = article['metadata']['tags']
    article['embedding'] = get_embedding(article_text + " " + " ".join(article_tags))

# Convert embeddings to NumPy array
article_embeddings = np.array([article['embedding'] for article in relevant_articles]).astype('float32')


In [10]:
# Create a base index for the relevant articles
article_index = faiss.IndexFlatL2(article_embeddings.shape[1])

# Create an IndexIDMap
article_index = faiss.IndexIDMap(article_index)

# IDs for the relevant articles
article_ids = np.array([i for i in range(len(relevant_articles))], dtype='int64')

# Add vectors and their IDs to the index
article_index.add_with_ids(article_embeddings, article_ids)

# Save the index to disk (optional)
faiss.write_index(article_index, "relevant_articles.index")

# Perform the search within the relevant articles
def query_relevant_articles(query, k=5):
    query_embedding = np.array(get_embedding(query)).astype('float32').reshape(1, -1)
    distances, indices = article_index.search(query_embedding, k)
    matching_articles = [relevant_articles[i] for i in indices[0]]
    return matching_articles

# Example query within the topic group
response_string = ""
matching_articles = query_relevant_articles(user_query)
for article in matching_articles:
    #print(f"Article Number: {article['article_number']}, Title: {article['title']}, Text: {article['text']}")
    response_string += f"Article Number: {article['article_number']}, Title: {article['title']}, Text: {article['text']}\n" + "\n"
    #response_string = response_string.replace("\n", "").strip()
   
print(response_string)


Article Number: Art. 83, Title: Öffentliche Ämter, Text: 1
Angestellte, die sich um ein öffentliches Amt bewerben wollen, melden dies der vorgesetzten Stelle. Eine Bewilligung ist erforderlich, sofern Arbeitszeit beansprucht wird. Vorbehalten bleiben Ämter mit Amtszwang. 
2
Die Bewilligung kann mit Auflagen zur Kompensation beanspruchter Arbeitszeit und zur Abgabe von Nebeneinnahmen verbunden werden.

Article Number: Art. 73, Title: Mitarbeiterinnen- und Mitarbeiterbeurteilung, Arbeitszeugnis, Text: 1
Die Angestellten haben Anspruch auf regelmässige Beurteilung von Leistung und Verhalten. 
2
Die Angestellten können jederzeit ein Zeugnis verlangen, das über die Art und die Dauer des Arbeitsverhältnisses sowie über ihre Leistungen und ihr Verhalten Auskunft gibt. 
3
Auf besonderes Verlangen der Angestellten hat sich das Zeugnis auf Angaben über die Art und Dauer des Arbeitsverhältnisses zu beschränken.

Article Number: Art. 68, Title: Schutz der Persönlichkeit, Text: 1
Die Arbeitgeberin 

In [19]:
from openai import OpenAI

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "system", "content": f"You are a helpful assistant. Who ansers the User Question on ONLY referencing the given Articles. {response_string} Decide if the Question can be relevantely answered using the Articles. Always Cite the Article you are refering to. If can confidentally anwer. Say 'I cant answer that'"},
    {"role": "user", "content": f"{user_query}"}
  ]
)

print(response.choices[0].message.content)

I cant answer that
